In [1]:
# 필요한 Import
import pandas as pd
import requests
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import time

In [ ]:
# --- 경로 설정 ---
# 모든 경로는 이 data_dir을 기준으로 생성됩니다.
data_dir = 'safebooru\data'
tmp_dir = os.path.join(data_dir, 'tmp') # 원본 이미지 다운로드 폴더
data_path = os.path.join(data_dir, 'all_data.csv')

In [3]:
# --- 병렬 처리 설정 ---
# I/O 작업이므로 워커 수를 높게 설정
WORKERS = 32
MAX_RETRIES = 3
RETRY_DELAY = 3  # 초

In [4]:
df = pd.read_csv(data_path)
print(f"전체 데이터 수: {len(df)}")

전체 데이터 수: 3020460


In [5]:
def download_image(row):
    # URL이 //로 시작하는 경우 http:를 붙여줌
    url = row['sample_url']
    if url.startswith('//'):
        url = 'http:' + url
    
    # 파일 확장자 결정 (기본 .jpg)
    ext = os.path.splitext(url)[1]
    if ext.lower() not in ['.jpg', '.png', '.gif']:
        ext = '.jpg'
        
    img_path = os.path.join(tmp_dir, f"{row['id']}{ext}")
    
    if os.path.exists(img_path):
        return {'id': row['id'], 'status': 'exist'}
        
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.get(url, timeout=15)
            if response.status_code == 200:
                with open(img_path, 'wb') as f:
                    f.write(response.content)
                return {'id': row['id'], 'status': 'success'}
            else:
                if attempt < MAX_RETRIES - 1:
                    time.sleep(RETRY_DELAY)
                    continue
                return {'id': row['id'], 'status': f'fail_status_{response.status_code}'}
        except Exception as e:
            if attempt < MAX_RETRIES - 1:
                time.sleep(RETRY_DELAY)
                continue
            return {'id': row['id'], 'status': 'fail_error'}

print("✅ 다운로드 함수 정의 완료")

✅ 다운로드 함수 정의 완료


In [ ]:
# DataFrame의 각 행을 리스트로 변환하여 map에 전달
rows_to_download = [row for _, row in df.iterrows()]

with ThreadPoolExecutor(max_workers=WORKERS) as executor:
    results = list(tqdm(executor.map(download_image, rows_to_download), total=len(rows_to_download), desc="이미지 다운로드"))

# (선택) 결과 요약
success_count = sum(1 for r in results if r['status'] == 'success')
exist_count = sum(1 for r in results if r['status'] == 'exist')
fail_count = sum(1 for r in results if 'fail' in r['status'])

print(f"\n✅ 다운로드 완료: 성공 {success_count}개, 이미 존재 {exist_count}개, 실패 {fail_count}개")

이미지 다운로드:   0%|          | 0/3020460 [00:00<?, ?it/s]